In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from xml.dom import minidom
from snowProfile import SnowProfile, SurfaceCondition, Layer, TempMeasurement
from whumpfData import WumphData

## Try other parser

In [2]:
import xml.etree.ElementTree as ET
file_path = "snowpits/wumph_pits/snowpits-25968-caaml.xml"


common_tag = '{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}'
root = ET.parse(file_path).getroot()


In [3]:
# measurementDirection
try:
    measurementDirection = next(root.iter(common_tag + 'SnowProfileMeasurements'), None).get('dir')
except AttributeError:
    measurementDirection = None

print(measurementDirection)


top down


In [4]:
hs_tag = common_tag + 'profileDepth'
        
try:
    hs = next(root.iter(hs_tag), None).text
    hs_units = next(root.iter(hs_tag), None).get('uom')
    HS = [float(hs), hs_units]
except AttributeError:
    HS = None


Surface Conditions
windLoading
penetrationFoot
penetrationSki

In [5]:
custom = root.iter(common_tag+'custom_Data')

for tag in custom:
    print(tag.text)

In [6]:
try:
    penFoot_val = next(root.iter(common_tag + 'penetrationFoot'), None).text
    penFoot_units = next(root.iter(common_tag + 'penetrationFoot'), None).get('uom')
    penFoot = [float(penFoot_val), penFoot_units]
except AttributeError:
    penFoot = None

In [7]:
# pitID
#  <caaml:locRef gml:id="location-nid-66387">

pitID_tag = common_tag + 'locRef'
gml_tag = '{http://www.opengis.net/gml}id'

try:
    pitID_str = next(root.iter(pitID_tag), None).attrib[gml_tag]
    pitID = pitID_str.split('-')[-1]
except AttributeError:
    pitID = None

print(pitID)

25968


In [8]:
#user = {
 #           'OperationID': None,
  #          'OperationName': None,
   #         'Professional': None,
    #        'ContactPersonID': None,
     #       'ContactPersonName': None,
      #      }   

user = {}


try:
    srcRef = root.iter(common_tag + 'srcRef')
except AttributeError:
    srcRef = None

print(srcRef)

if srcRef is not None:
    for prop in srcRef:
        print("tag: ",prop.tag)
        print("attrib: ",prop.attrib)
        print("text: ",prop.text)
        for sub_prop in prop:
            print("sub_tag: ",sub_prop.tag)
            print("sub_attrib: ",sub_prop.attrib)
            print("sub_text: ",sub_prop.text)
            for sub_sub_prop in sub_prop:
                print("sub_sub_tag: ",sub_sub_prop.tag)
                print("sub_sub_attrib: ",sub_sub_prop.attrib)
                print("sub_sub_text: ",sub_sub_prop.text)
                for sub_sub_sub_prop in sub_sub_prop:
                    print("sub_sub_sub_tag: ",sub_sub_sub_prop.tag)
                    print("sub_sub_sub_attrib: ",sub_sub_sub_prop.attrib)
                    print("sub_sub_sub_text: ",sub_sub_sub_prop.text)

if srcRef is not None:
    for prop in srcRef:
        for sub_prop in prop:
            if sub_prop.tag.endswith('Operation'):
                user['OperationID'] = sub_prop.attrib['gml:id']
                for sub_sub_prop in sub_prop:
                    if sub_sub_prop.tag.endswith('name'):
                        user['OperationName'] = sub_sub_prop.text
                    if sub_sub_prop.tag.endswith('contactPerson'):
                        user['ContactPersonID'] = sub_sub_prop.attrib['gml:id']
                        for sub_sub_sub_prop in sub_sub_prop:
                            if sub_sub_sub_prop.tag.endswith('name'):
                                user['ContactPersonName'] = sub_sub_sub_prop.text
            elif sub_prop.tag.endswith('Person'):
                user['ContactPersonID'] = sub_prop.attrib['gml:id']
                for sub_sub_prop in sub_prop:
                    if sub_sub_prop.tag.endswith('name'):
                        user['ContactPersonName'] = sub_sub_prop.text



print(user)


tag:  {http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}srcRef
attrib:  {}
text:  
    
sub_tag:  {http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}Operation
sub_attrib:  {'{http://www.opengis.net/gml}id': 'SnowPilot-Group-181'}
sub_text:  
      
sub_sub_tag:  {http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}name
sub_sub_attrib:  {}
sub_sub_text:  Colorado Avalanche Information Center
sub_sub_tag:  {http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}contactPerson
sub_sub_attrib:  {'{http://www.opengis.net/gml}id': 'SnowPilot-User-3633'}
sub_sub_text:  
        
sub_sub_sub_tag:  {http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}name
sub_sub_sub_attrib:  {}
sub_sub_sub_text:  Jason Konigsberg
{}


In [9]:
# Custom Data

try:
    customData = root.iter(common_tag+'customData')
except AttributeError:
    customData = None

for prop in customData:
    for sub_prop in prop:
        if(sub_prop.tag.endswith('whumpfData')):
            isWhumpfData = True
            whumpfData = sub_prop

        if(sub_prop.tag.endswith('windLoading')):
            windLoading = sub_prop.text

if(whumpfData != None):
    for prop in whumpfData:
        if(prop.tag.endswith('whumpfCracking')):
            whumpfCracking = prop.text
        if(prop.tag.endswith('whumpfNoCracking')):
            whumpfNoCracking = prop.text
        if(prop.tag.endswith('crackingNoWhumpf')):
            crackingNoWhumpf = prop.text
        if(prop.tag.endswith('whumpfNearPit')):
            whumpfNearPit = prop.text
        if(prop.tag.endswith('whumpfDepthWeakLayer')):
            whumpfDepthWeakLayer = prop.text
        if(prop.tag.endswith('whumpfTriggeredRemoteAva')):
            whumpfTriggeredRemoteAva = prop.text
        if(prop.tag.endswith('whumpfSize')):
            whumpfSize = prop.text
            
            

        print(prop.tag)
        print(prop.text)




print(windLoading)


{http://www.snowpilot.org/Schemas/caaml}whumpfCracking
true
{http://www.snowpilot.org/Schemas/caaml}whumpfNoCracking
false
{http://www.snowpilot.org/Schemas/caaml}crackingNoWhumpf
false
{http://www.snowpilot.org/Schemas/caaml}whumpfNearPit
true
{http://www.snowpilot.org/Schemas/caaml}whumpfDepthWeakLayer
true
{http://www.snowpilot.org/Schemas/caaml}whumpfTriggeredRemoteAva
false
{http://www.snowpilot.org/Schemas/caaml}whumpfSize
None
no


In [12]:
    # Layers
layers = root.iter(common_tag + 'Layer')

for layer in layers:
        for prop in layer.iter():
            print(prop.tag)
            print(prop.text)

{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}Layer

          
{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}depthTop
0
{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}thickness
1
{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}grainFormPrimary
FCsf
{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}grainSize

            
{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}Components

              
{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}avg
0.5
{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}hardness
4F
{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}Layer

          
{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}depthTop
1
{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}thickness
17
{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}grainFormPrimary
FC
{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}grainSize

            
{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}Components

              
{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}avg
0.3
{http://ca

In [14]:
    # Layers
layers = root.iter(common_tag + 'Layer')

for layer in layers:

        layer_obj = Layer()
        for prop in layer.iter():
            if prop.tag.endswith('depthTop'):
                depthTop_val = prop.text
                depthTop_units = prop.get('uom')
                depthTop=[depthTop_val,depthTop_units]
                layer_obj.set_depthTop(depthTop)
            if prop.tag.endswith('thickness'):
                thickness_val = prop.text
                thickness_units = prop.get('uom')
                thickness=[thickness_val,thickness_units]
                layer_obj   .set_thickness(thickness)
            if prop.tag.endswith('grainFormPrimary'):
                grainFormPrimary = prop.text
                layer_obj.grainFormPrimary.set_grainForm(grainFormPrimary)
            if prop.tag.endswith('grainFormSecondary'):
                grainFormSecondary = prop.text
                layer_obj.grainFormSecondary.set_grainForm(grainFormSecondary)
            if prop.tag.endswith('grainSize'):
                grainSize_units = prop.get('uom')
                for sub_prop in prop:
                    for sub_sub_prop in sub_prop:
                        if sub_sub_prop.tag.endswith('avg'):
                            grainSizeAvg = sub_sub_prop.text
                            grainSizeAvg = [float(grainSizeAvg), grainSize_units]
                            layer_obj.grainFormPrimary.set_grainSizeAvg(grainSizeAvg)
                        if sub_sub_prop.tag.endswith('max'):
                            grainSizeMax = sub_sub_prop.text
                            grainSizeMax = [float(grainSizeMax), grainSize_units]
                            layer_obj.grainFormPrimary.set_grainSizeMax(grainSizeMax)






            if prop.tag.endswith('hardness'):
                hardness = prop.text
                layer_obj.set_hardness1(hardness)
            if prop.tag.endswith('wetness'):
                wetness = prop.text
                layer_obj.set_wetness(wetness)
            if prop.tag.endswith('layerOfConcern'):
                layerOfConcern = prop.text
                layer_obj.set_layerOfConcern(layerOfConcern)
        print(layer_obj)




	 depthTop: ['0', 'cm']
	 thickness: ['1', 'cm']
	 grainFormPrimary: 
		 grainForm: FCsf
		 grainSizeAvg: [0.5, 'mm']
		 grainSizeMax: None
		 basicGrainClass: FC
		 subGrainClass: FCsf
	 grainFormSecondary: 
		 grainForm: None
		 grainSizeAvg: None
		 grainSizeMax: None
		 basicGrainClass: None
		 subGrainClass: None
	 density: None
	 wetness: None
	 layerOfConcern: None
	 comments: None

	 depthTop: ['1', 'cm']
	 thickness: ['17', 'cm']
	 grainFormPrimary: 
		 grainForm: FC
		 grainSizeAvg: [0.3, 'mm']
		 grainSizeMax: None
		 basicGrainClass: FC
		 subGrainClass: None
	 grainFormSecondary: 
		 grainForm: None
		 grainSizeAvg: None
		 grainSizeMax: None
		 basicGrainClass: None
		 subGrainClass: None
	 density: None
	 wetness: None
	 layerOfConcern: None
	 comments: None

	 depthTop: ['18', 'cm']
	 thickness: ['6', 'cm']
	 grainFormPrimary: 
		 grainForm: RGxf
		 grainSizeAvg: [0.3, 'mm']
		 grainSizeMax: None
		 basicGrainClass: RG
		 subGrainClass: RGxf
	 grainFormSecondary: 
		 g